# 0.2.0 Load Pre-Calculated Heatmap

In [1]:
from clustergrammer2 import net

>> clustergrammer2 backend version 0.5.5


In [2]:
net.viz

{'row_nodes': [],
 'col_nodes': [],
 'links': [],
 'mat': [],
 'cat_colors': {'row': {}, 'col': {}}}

In [3]:
net.dat

{'nodes': {'row': [], 'col': []},
 'mat': [],
 'node_info': {'row': {'ini': [],
   'clust': [],
   'rank': [],
   'info': [],
   'cat': [],
   'value': []},
  'col': {'ini': [],
   'clust': [],
   'rank': [],
   'info': [],
   'cat': [],
   'value': []}}}

In [4]:
viz_dict = net.load_json_to_dict('data/processed/pre-calc-viz.json')
viz_dict.keys()

dict_keys(['row_nodes', 'col_nodes', 'links', 'mat', 'cat_colors', 'views'])

In [5]:
net.viz = viz_dict

In [6]:
net.widget()

ExampleWidget(network='{"row_nodes": [{"name": "Bc1", "ini": 250, "clust": 48, "rank": 203, "rankvar": 217, "g…